In [1]:
from pathlib import Path
from os import path
import datetime
import glob
import pandas as pd
inv_folder = '../PriceScript_20220424/inventories'

# Comparison

In [2]:
def comparison(df1, df2, merge_on, col):
    delta=0.1
        
    print('NEW  Entries     : ',len(df1))
    print('OLD  Entries     : ',len(df2))
    
    comp = df1.merge(right=df2, how='outer', on= merge_on, suffixes=('_new', '_old'))
    comp['Category'] = pd.NA
    comp.loc[comp[col + '_new'] - comp[col + '_old'] > delta , 'Category'] = 'Price Increased'
    comp.loc[comp[col + '_old'] - comp[col + '_new'] > delta, 'Category'] = 'Price Decreased'
    comp.loc[abs(comp[col + '_new'] - comp[col + '_old']) <= delta, 'Category'] = 'Same Price'
    comp.loc[(~comp[col + '_new'].isna()) & (comp[col + '_old'].isna()), 'Category'] = 'Items Added'
    comp.loc[(comp[col + '_new'].isna()) & (~comp[col + '_old'].isna()), 'Category'] = 'Items Removed'
    
    
    print('Increase in Price: ', len(comp[comp[col + '_new'] > (comp[col + '_old'] +  delta)]))
    print('Decrease in Price: ', len(comp[comp[col + '_new'] < (comp[col + '_old'] -  delta)]))
    print('Same Price       : ', len(comp[abs(comp[col + '_new'] - comp[col + '_old']) <=  delta]))
    print('Increase in Parts: ', len(comp[(~comp[col + '_new'].isna()) & (comp[col + '_old'].isna())]))
    print('Decrease in Parts: ', len(comp[(comp[col + '_new'].isna()) & (~comp[col + '_old'].isna())]))
    
    detail = comp.loc[comp['Category'] != 'Same Price']
    group = comp.groupby('Category')
    Fcount = group[['Category']].count()
    Fcount.rename(columns={'Category': 'Count'}, inplace=True)
    Fcount.reset_index(inplace=True)
    
    return Fcount, detail

# Date Extract

In [3]:
def dateextract(f):
    b= path.basename(f)
    c= b.split('.')[0].split('_')[1]
    d = datetime.datetime.strptime(c, "%Y%m%d").date()
    return d

# Read File and Add Date Extracted

In [4]:
def read_df(f, sep=',', encoding=None, encoding_errors='strict', escapechar=None):
    df = pd.read_csv(f,sep=sep, encoding=encoding, encoding_errors=encoding_errors, escapechar=escapechar)
    dt = dateextract(f)
    df['Date'] = dt
    return df

# Protocols for each warehouse


### Brock

In [7]:
df1=read_df(f'{inv_folder}/brock_20220807.csv')
df2=read_df(f'{inv_folder}/brock_20220804.csv')
merge_on= ['sku']
col='prices'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'Brock'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/brock_Summary.xlsx')
b.to_excel('results/brock_Detail.xlsx')
result=[a]

NEW  Entries     :  11692
OLD  Entries     :  11722
Increase in Price:  0
Decrease in Price:  0
Same Price       :  11690
Increase in Parts:  2
Decrease in Parts:  32


### Burco

In [8]:
df1=read_df(f'{inv_folder}/burco_20220608.csv')
df2=read_df(f'{inv_folder}/burco_20220805.csv')
merge_on= ['vendor_sku']
col='Cost'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'Burco'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/burco_Summary.xlsx')
b.to_excel('results/burco_Detail.xlsx')
result.append(a)

NEW  Entries     :  4259
OLD  Entries     :  4259
Increase in Price:  0
Decrease in Price:  0
Same Price       :  4259
Increase in Parts:  0
Decrease in Parts:  0


### Jante

In [10]:
df1=read_df(f'{inv_folder}/jante_20220807.csv')
df2=read_df(f'{inv_folder}/jante_20220804.csv')
merge_on= ['LC' , 'vendor_sku']
col='Cost'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'Jante'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/jante_Summary.xlsx')
b.to_excel('results/jante_Detail.xlsx')
result.append(a)

NEW  Entries     :  6275
OLD  Entries     :  6275
Increase in Price:  0
Decrease in Price:  0
Same Price       :  6273
Increase in Parts:  2
Decrease in Parts:  0


### Keystone

In [11]:
df1=read_df(f'{inv_folder}/keystone_20220808.csv')
df2=read_df(f'{inv_folder}/keystone_20220804.csv')
merge_on= ['VCPN']
col='Cost'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'Keystone'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/Keystone_Summary.xlsx')
b.to_excel('results/Keystone_Detail.xlsx')
result.append(a)

NEW  Entries     :  159263
OLD  Entries     :  159463
Increase in Price:  1072
Decrease in Price:  840
Same Price       :  157049
Increase in Parts:  292
Decrease in Parts:  492


### Motorstate

In [13]:
df1=read_df(f'{inv_folder}/motorstate_20220807.csv')
df2=read_df(f'{inv_folder}/motorstate_20220803.csv')
merge_on= ['Brand' , 'PartNumber']
col='Cost'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'Motorstate'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/motorstate_Summary.xlsx')
b.to_excel('results/motorstate_Detail.xlsx')
result.append(a)

NEW  Entries     :  107499
OLD  Entries     :  107532
Increase in Price:  491
Decrease in Price:  625
Same Price       :  106219
Increase in Parts:  164
Decrease in Parts:  197


### NPW

In [14]:
df1=read_df(f'{inv_folder}/npw_20220808.csv')
df2=read_df(f'{inv_folder}/npw_20220804.csv')
merge_on= ['LineCode' , 'PartN']
col='Cost'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'NPW'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/npw_Summary.xlsx')
b.to_excel('results/npw_Detail.xlsx')
result.append(a)

NEW  Entries     :  310652
OLD  Entries     :  383647
Increase in Price:  26
Decrease in Price:  32
Same Price       :  310279
Increase in Parts:  315
Decrease in Parts:  73310


### PA

In [15]:
df1=read_df(f'{inv_folder}/pa_20220808.csv', encoding='unicode_escape')
df2=read_df(f'{inv_folder}/pa_20220805.csv', encoding='unicode_escape')
merge_on= ['Line' , 'Part']
col='Price'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'PA'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/pa_Summary.xlsx')
b.to_excel('results/pa_Detail.xlsx')
result.append(a)

NEW  Entries     :  524100
OLD  Entries     :  525098
Increase in Price:  308
Decrease in Price:  849
Same Price       :  519760
Increase in Parts:  3182
Decrease in Parts:  4180


### PFG

In [16]:
df1=read_df(f'{inv_folder}/pfg_20220808.txt', sep='\t', escapechar = '\\', encoding_errors='ignore')
df2=read_df(f'{inv_folder}/pfg_20220805.txt', sep='\t', escapechar = '\\', encoding_errors='ignore')
merge_on = ['SKU', 'BRAND']
col='COST'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'PFG'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/pfg_Summary.xlsx')
b.to_excel('results/pfg_Detail.xlsx')
result.append(a)

NEW  Entries     :  82896
OLD  Entries     :  82901
Increase in Price:  0
Decrease in Price:  0
Same Price       :  82851
Increase in Parts:  45
Decrease in Parts:  50


### RSL

In [17]:
df1=read_df(f'{inv_folder}/rsl_20220807.csv')
df2=read_df(f'{inv_folder}/rsl_20220804.csv')
merge_on= ['SKU', 'Brand']
col='BSAP Cost'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'RSL'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/rsl_Summary.xlsx')
b.to_excel('results/rsl_Detail.xlsx')
result.append(a)

NEW  Entries     :  5592
OLD  Entries     :  5592
Increase in Price:  0
Decrease in Price:  0
Same Price       :  5593
Increase in Parts:  0
Decrease in Parts:  0


### Sunbelt

In [ ]:
df1=read_df(f'{inv_folder}/sunbelt_20220601.csv')
df2=read_df(f'{inv_folder}/sunbelt_20220531.csv')
merge_on= ['SKU']
col='Price'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'SUNBELT'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/sunbelt_Summary.xlsx')
b.to_excel('results/sunbelt_Detail.xlsx')
result.append(a)

### Simple Tire

In [19]:
### Simple Tire
df1=read_df(f'{inv_folder}/simpletire_20220807.csv')
df2=read_df(f'{inv_folder}/simpletire_20220804.csv')
merge_on= ['Product ID']
col='Price'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'SIMPLE TIRE'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/simpletire_Summary.xlsx')
b.to_excel('results/simpletire_Detail.xlsx')
result.append(a)

NEW  Entries     :  43827
OLD  Entries     :  43927
Increase in Price:  5871
Decrease in Price:  4596
Same Price       :  33119
Increase in Parts:  240
Decrease in Parts:  340


### Dorman

In [20]:
df1=read_df(f'{inv_folder}/dorman_20220807.csv')
df2=read_df(f'{inv_folder}/dorman_20220804.csv')
merge_on= ['599-610']

df1['Price']=  0
df2['Price']=  0
col= 'Price'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'DORMAN'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/dorman_Summary.xlsx')
b.to_excel('results/dorman_Detail.xlsx')
result.append(a)

NEW  Entries     :  80119
OLD  Entries     :  80041
Increase in Price:  0
Decrease in Price:  0
Same Price       :  80041
Increase in Parts:  78
Decrease in Parts:  0


### OE Wheels

In [21]:
df1=read_df(f'{inv_folder}/oe_20220807.csv')
df2=read_df(f'{inv_folder}/oe_20220804.csv')
merge_on= ['UPC']

df1['Price']=  0
df2['Price']=  0
col= 'Price'

a,b = comparison(df1, df2, merge_on, col)
a['Warehouse']= 'OE WHEELS'
a['Latest Date']= df1.loc[0, 'Date']
a['Previous Date']= df2.loc[0, 'Date']

a.to_excel('results/oe_Summary.xlsx')
b.to_excel('results/oe_Detail.xlsx')
result.append(a)

NEW  Entries     :  766
OLD  Entries     :  766
Increase in Price:  0
Decrease in Price:  0
Same Price       :  766
Increase in Parts:  0
Decrease in Parts:  0


# Result:

In [23]:
r = pd.concat(result, ignore_index=True)
r.to_excel('Inventory Summary_testing.xlsx', index=False)
r

,Category,Count,Warehouse,Latest Date,Previous Date
0,Items Added,2,Brock,2022-08-07,2022-08-04
1,Items Removed,32,Brock,2022-08-07,2022-08-04
2,Same Price,11690,Brock,2022-08-07,2022-08-04
3,Same Price,4259,Burco,2022-06-08,2022-08-05
4,Items Added,2,Jante,2022-08-07,2022-08-04
5,Same Price,6273,Jante,2022-08-07,2022-08-04
6,Items Added,292,Keystone,2022-08-08,2022-08-04
7,Items Removed,492,Keystone,2022-08-08,2022-08-04
8,Price Decreased,840,Keystone,2022-08-08,2022-08-04
9,Price Increased,1082,Keystone,2022-08-08,2022-08-04
